In [ ]:
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
import torch

# - Rockpool imports
from rockpool.parameters import Constant
from rockpool.timeseries import TSEvent
from rockpool.transform import quantize_methods as q

# - Modules required to interface and deploy models to the Xylo-IMU HDK
from rockpool.devices.xylo import find_xylo_hdks
from rockpool.devices.xylo.syns61201 import XyloSamna, config_from_specification, mapper, XyloSim
#from rockpool.devices.xylo.syns63300 import XyloSamna, config_from_specification, mapper, XyloSim

import sys
sys.path.append("../training-pipeline/lib")
from data_loading import load_data
from xylo_net_adaptation import net_from_params, adapt_network
import xylo_networks as xylo_networks

## Data Loading and Model Restoration

In [11]:
# Configuration
model_path = "../nni_experiments/xylo/final/xylo_7cb_ff_deep_deep_res"

def load_data_from_model_path(model_path):
    # Obtain training metadata
    training_metadata = None
    with open(f"{model_path}/training_metadata.json") as f:
        training_metadata = json.load(f)

    # Load dataset
    input_params = training_metadata["input_params"]

    train_dl, val_dl, test_dl = load_data(**input_params)
    return train_dl, val_dl, test_dl, input_params


def load_model_from_path(model_path, dt):
    # Obtain neuron parameters
    model_metadata = None
    with open(f"{model_path}/model_metadata.json") as f:
        model_metadata = json.load(f)

    neuron_parameters = {
        "tau_mem": model_metadata["params"]["tau_mem"],
        "tau_syn": model_metadata["params"]["tau_syn"],
        "bias": Constant(0.0),
        "threshold": Constant(1.0),
        "dt": dt,
    }

    # Build network from model parameters
    model_params = None
    with open(f"{model_path}/model_params") as f:
        model_params = json.load(f)

    net = net_from_params(model_params, neuron_parameters)

    print(f"Built network: \n\t{net}")

    # Load network parameters
    net.load(f"{model_path}/model_params")

    # Adapt network to be compatible with and deployable on Xylo
    adapt_network(net, max_synapses=62)

    return net

train_dl, val_dl, test_dl, input_params = load_data_from_model_path(model_path)

## Model Evaluation

In [ ]:
# Load network from model path
net = load_model_from_path(model_path, input_params["dt"])

# Evaluate network on the full test set
net.eval()
ds = test_dl.dataset
with torch.no_grad():
    output, _, _ = net(ds.x)
    preds = output.sum(dim=1).argmax(dim=1)
    print(f"Final accuracy: {torch.round(torch.mean(preds == ds.y, dtype=float)*100,decimals=2)}%")

# Calculate confusion matrix
cm = confusion_matrix(ds.y, preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

# Print the classification report
print(classification_report(ds.y, preds))

In [ ]:
# Plot the network activity for a specific sample
net.eval()
ds = test_dl.dataset
i = 1
with torch.no_grad():
    output, _, b = net(ds.x[i])

    spks = torch.sum(output,dim=1)
    print(f"Neural Activity (spikes per neuron): {spks.numpy()[0]}")
    print(f"Expected Label: {int(ds.y[i])}")
    print(f"Predicted Label: {spks.argmax()}")

    TSEvent.from_raster(
                output[0].detach().numpy(),
                dt=5e-2,
            ).plot(marker="|", s=8)
    plt.plot(0, ds.y[i], '>', ms=20)
    plt.tight_layout()
    plt.show()

## Quantization Tuning and Hardware Config Generation

In [ ]:
# Load network from model path
net = load_model_from_path(model_path, input_params["dt"])

# Convert network to spec
spec = mapper(net.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
spec_Q = copy.deepcopy(spec)

# Use quantization tuned specification object, if available
try:
    with open(f"{model_path}/xylo_qtuned_params", "rb") as f:
        params_Q = pickle.load(f)
        spec_Q.update(params_Q)
    print("Model tuning applied")
except FileNotFoundError:
    # Quantize the parameters
    spec_Q.update(q.global_quantize(**spec_Q))
    print("No model tuning available")

#TODO: add reference to github issue
if spec_Q["dash_syn"][0] == 0:
    spec_Q["dash_syn"] += 1
if spec_Q["dash_syn_out"][0] == 0:
    spec_Q["dash_syn_out"] += 1

# Convert spec to Xylo configuration
config, is_valid, m = config_from_specification(**spec_Q)
if not is_valid:
    raise ValueError(f"Error detected in spec:\n{m}")
else:
    print("ok")

# Build XyloSim from the config
mod = XyloSim.from_config(config, dt = input_params["dt"])
print(mod)

## Quantized Network Analysis

In [ ]:
# Display the quantized network weights
plt.figure(figsize=(20,20))
plt.subplot(1, 3, 1)
plt.imshow(abs(spec['weights_in'].T), cmap="hot")
plt.title('$W_{in}$')

plt.subplot(1, 3, 2)
plt.imshow(abs(spec['weights_rec'].T), cmap="hot")
plt.title('$W_{rec}$')

plt.subplot(1, 3, 3)
plt.imshow(abs(spec['weights_out'].T), cmap="hot")
plt.title('$W_{out}$');



In [ ]:
# Evaluate the XyloSim network on the full test set
ds = test_dl.dataset
preds = []
scores = []
for x,y in zip(ds.x,ds.y):
    output, _, rec = mod(x.numpy())
    pred = np.sum(output,axis=0)
    preds.append(np.argmax(pred))
    scores.append(np.argmax(pred) == y.numpy())

acc = np.sum(scores)/len(scores)

print(f"Final accuracy: {np.round(acc, decimals=4)*100}%")

# Calculate confusion matrix
cm = confusion_matrix(ds.y, preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

# Print the classification report
print(classification_report(ds.y, preds))

In [ ]:
# Plot the XyloSim network activity for a specific sample
ds = test_dl.dataset
i = 1

output, _, rec = mod(ds.x[i].numpy(),record=True)

spks = np.sum(output,axis=0)
pred = np.argmax(spks)
print(f"Readout layer activity (spikes): {spks}")
print(f"Label: {int(ds.y[i])}")
print(f"Prediction: {pred}")

TSEvent.from_raster(
            output,
            dt=input_params["dt"],
        ).plot(marker="|", s=8)
plt.plot(0.01, ds.y[i], '>', ms=10, color='g')
plt.plot(0.01, pred, '>', ms=10, color='r')
plt.tight_layout()
plt.show()

times = np.arange(output.shape[0]) * input_params["dt"]
plt.plot(times, rec['Vmem_out'])
plt.legend(range(0,7))
plt.show()

## Xylo Hardware Deploy

In [ ]:
# DISCLAIMER:THE FOLLOWING CODE IS FOR A XYLO IMU BOARD, NOT THE XYLO AUDIO ADDRESSED BY THIS THESIS
devices, modules, vers = find_xylo_hdks()
print(devices, vers)

found_device = None
for i,d in enumerate(devices):
    if vers[i] == "syns63300":
        found_device = d
        found_device_module = modules[i]

if found_device == None:
    raise ValueError("No Xylo found")

print(f'Setting Xylo main clock to {found_device_module.xylo_imu_devkit_utils.set_xylo_core_clock_freq(found_device, 15)} MHz')
mod_hdk = XyloSamna(found_device, config, input_params["dt"],power_frequency=20.)

In [ ]:
# Evaluate the Xylo network on the full test set
ds = test_dl.dataset
preds = []
scores = []
for i,(x,y) in list(enumerate(zip(ds.x,ds.y)))[0:100]:
    output, _, rec = mod_hdk.evolve(x.numpy().astype(int),record=True)
    pred = np.sum(output,axis=0)
    preds.append(np.argmax(pred))
    scores.append(np.argmax(pred) == y.numpy())

acc = np.sum(scores)/len(scores)

print(f"Final accuracy: {np.round(acc, decimals=4)*100}%")

# Calculate confusion matrix
cm = confusion_matrix(ds.y[0:100], preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

# Print the classification report
print(classification_report(ds.y[0:100], preds))

In [ ]:
# Plot the XyloSim network activity for a specific sample
ds = test_dl.dataset
i = 1

output, _, rec = mod_hdk.evolve(ds.x[i].numpy().astype(int),record=True,record_power=True)

spks = np.sum(output,axis=0)
pred = np.argmax(spks)
print(f"Readout layer activity (spikes): {spks}")
print(f"Label: {int(ds.y[i])}")
print(f"Prediction: {pred}")

TSEvent.from_raster(
            output,
            dt=input_params["dt"],
        ).plot(marker="|", s=8)
plt.plot(0.01, ds.y[i], '>', ms=10, color='g')
plt.plot(0.01, pred, '>', ms=10, color='r')
plt.tight_layout()
plt.show()

times = np.arange(output.shape[0]) * input_params["dt"]
plt.plot(times, rec['Vmem_out'])
plt.legend(range(0,7))
plt.show()

# Show active power consumption
print(f"Active IO power:\t{np.mean(rec['io_power']) * 1e6}µW\nSNN + IMU IF core:\t{np.mean(rec['core_power']) * 1e6}µW")

In [ ]:
# Show idle power consumption (no evolution)
from time import sleep

mod_hdk._power_buf.clear_events()
sleep(5.)
power = mod_hdk._power_buf.get_events()

power_idle = ([], [])

for p in power:
    power_idle[p.channel].append(p.value)

idle_power_per_channel = np.mean(np.stack(power_idle), axis = 1)

print(f'Idle IO power:\t\t{idle_power_per_channel[0] * 1e6}µW\nSNN + IMU IF core:\t{idle_power_per_channel[1]*1e6}µW')